In [101]:

import pandas as pd
from dateutil.relativedelta import relativedelta
import ta
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [102]:
def restructure_df(df):
    """
    Restructures the input DataFrame to move 'Quarter' and 'Fiscal year'
    values into the 'Metric' column.

    Args:
        df (pd.DataFrame): Input DataFrame with columns 'Ticker', 'Date', 'Metric',
                           'Value', 'Quarter', and 'Fiscal year'.

    Returns:
        pd.DataFrame: Restructured DataFrame with columns 'Ticker', 'Date', 'Metric', 'Value'.
    """

    # Melt the DataFrame to combine 'Quarter' and 'Fiscal year' into 'Metric'
    df_melted = pd.melt(
        df,
        id_vars=['Ticker', 'Date', 'Metric', 'Value'],
        value_vars=['Quarter', 'Fiscal year'],
        var_name='New_Metric',
        value_name='New_Value'
    )

    # Create a new DataFrame from the original 'Metric' and 'Value' columns
    df_base = df_melted[['Ticker', 'Date', 'Metric', 'Value']].dropna()

    # Create a new DataFrame from the 'New_Metric' and 'New_Value' columns
    df_new = df_melted[['Ticker', 'Date', 'New_Metric', 'New_Value']].rename(
        columns={'New_Metric': 'Metric', 'New_Value': 'Value'}
    ).dropna()

    # Concatenate the two DataFrames and return the result
    return pd.concat([df_base, df_new], ignore_index=True)[['Ticker', 'Date', 'Metric', 'Value']]


In [103]:
def compute_quarter_metrics(df, dates):
    df = df.copy()
    df["Date"] = pd.to_datetime(df["Date"].astype(str).str[:10], errors="coerce")
    results = []

    for ticker, df_ticker in df.groupby("Company"):
        df_ticker = df_ticker.sort_values("Date")
        # Precompute indicators
        macd = ta.trend.MACD(close=df_ticker["Close"], window_slow=60, window_fast=30, window_sign=30)
        rsi = ta.momentum.RSIIndicator(close=df_ticker["Close"], window=60).rsi()
        df_ticker["MACD"] = macd.macd()
        df_ticker["MACD_Signal"] = macd.macd_signal()
        df_ticker["MACD_Hist"] = macd.macd_diff()
        df_ticker["RSI_14"] = rsi

        for end_date in df_ticker.loc[df_ticker["Date"].dt.strftime("%m-%d").isin(dates), "Date"].unique():
            if pd.isnull(end_date): continue
            start_date = end_date.replace(day=1) - relativedelta(months=2)
            df_period = df_ticker[(df_ticker["Date"] >= start_date) & (df_ticker["Date"] <= end_date)]
            if df_period.empty: continue

            metrics = {
                "Ticker": ticker,
                "Date": end_date,
                "ClosePrice": df_period.iloc[-1]["Close"],
                "MinPrice": df_period["Close"].min(),
                "MaxPrice": df_period["Close"].max(),
                "StdPrice": df_period["Close"].std(),
                "MeanPrice": df_period["Close"].mean(),
                "MedianPrice": df_period["Close"].median(),
                "MinVolume": df_period["Volume"].min(),
                "MaxVolume": df_period["Volume"].max(),
                "StdVolume": df_period["Volume"].std(),
                "MeanVolume": df_period["Volume"].mean(),
                "MedianVolume": df_period["Volume"].median(),
                "SumDividends": df_period["Dividends"].sum(),
                "MeanDividends": df_period["Dividends"].replace(0, np.nan).mean() or 0,
                "CountDividends": (df_period["Dividends"] != 0).sum(),
                "RSI_14": df_period["RSI_14"].iloc[-1],
                "MACD": df_period["MACD"].iloc[-1],
                "MACD_Signal": df_period["MACD_Signal"].iloc[-1],
                "MACD_Hist": df_period["MACD_Hist"].iloc[-1]
            }
            results.append(metrics)
    return pd.DataFrame(results)

In [104]:
def clean_numeric_column(series):
    # Convert all values to string and strip whitespace.
    s = series.astype(str).str.strip()
    # Remove dollar sign (the regex ensures that only the $ symbol is removed).
    s = s.str.replace(r'\$', '', regex=True)
    # Remove commas.
    s = s.str.replace(',', '', regex=True)
    # Replace values that are exactly "-" or empty with "0"
    s = s.replace({'-': '0', '': '0'})
    # Convert to numeric (this will preserve negative numbers)
    return pd.to_numeric(s, errors='coerce')

In [105]:
import pandas as pd
import numpy as np

def calc_quarterly_pct_diff(df, ticker_col='ticker', date_col='date', lags=[1, 4]):
    """
    Calculate the percentage change for each numeric column by ticker between periods 
    for given lags. For each numeric column, it computes the percent change for each lag 
    and appends new columns named "<col>_pct_diff_<lag>".
    
    The first row for each ticker is NaN for each lag since there's no previous period.

    Args:
        df (pd.DataFrame): The input DataFrame.
        ticker_col (str): Column name for ticker identifiers.
        date_col (str): Column name for date/time variable.
        lags (list): List of lag values (in periods) for which to compute the percent change.

    Returns:
        pd.DataFrame: A copy of 'df' with new percentage change columns appended.
    """
    # Sort by ticker/date to ensure correct chronological order
    df = df.sort_values(by=[ticker_col, date_col]).copy()

    # Identify numeric columns (exclude ticker/date from numeric, if present)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.difference([ticker_col, date_col])

    # Build a dict of new columns; each key corresponds to a new Series
    new_cols = {}

    # For each numeric column, compute percent change for each specified lag
    for col in numeric_cols:
        for lag in lags:
            new_col_name = f"{col}_pct_diff_{lag}"
            new_cols[new_col_name] = df.groupby(ticker_col)[col].pct_change(periods=lag)

    # Convert the dict to a DataFrame, aligning on the original index, then concat once
    df_new = pd.DataFrame(new_cols, index=df.index)
    df = pd.concat([df, df_new], axis=1)

    return df

___

In [106]:
prices = pd.read_csv("sp500_prices.csv", index_col=0, parse_dates=True, low_memory=False)
prices.rename(columns={"index": "Date"}, inplace=True)
prices.tail(1)

C:\Users\Dell\AppData\Local\Temp\ipykernel_36764\659451341.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  prices = pd.read_csv("sp500_prices.csv", index_col=0, parse_dates=True, low_memory=False)


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Company
6248600,2025-05-12 00:00:00-04:00,161.029999,164.589996,160.679993,161.919998,2810554.0,0.0,0.0,ZTS


In [107]:
names = pd.read_csv("sp500_names.csv")
names.tail(1)

,Unnamed: 0,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
503,503,ZTS,Zoetis,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952


In [ ]:
financials = pd.read_csv("sp500_financials_new0.csv")
financials_clean = financials[~financials['Period Ending'].str.contains('Quarters')]
financials_clean = financials_clean[financials_clean['Fiscal Quarter'].str.contains('Q')]
financials_clean = financials_clean[financials_clean['Fiscal Quarter']!='Current']
financials_clean.fillna(0, inplace=True)
financials_clean['Date'] = pd.to_datetime(financials_clean['Period Ending'].str[8:], format='%b %d, %Y', errors='coerce')
financials_clean.drop(columns=['Period Ending'], inplace=True)
financials_clean.drop(columns=['Unnamed: 0'], inplace=True)
financials_clean[['Quarter', 'Fiscal year']] = financials_clean['Fiscal Quarter'].str.split(' ', expand=True)
financials_clean['Quarter'] = financials_clean['Quarter'].str[1:].astype(int)        # Remove 'Q' and convert to int
financials_clean['Fiscal year'] = financials_clean['Fiscal year'].astype(int)        # Convert year to int
financials_clean.drop(columns=['Fiscal Quarter'], inplace=True)
financials_fiscal = restructure_df(financials_clean)
financials_fiscal.columns = ['Ticker', 'Date', 'Variable', 'Value']
financials_fiscal.tail(1)

,Ticker,Date,Variable,Value
12882307,WELL,2020-06-30,Fiscal year,2020


In [109]:
dates = financials_fiscal['Date'].astype('str').str[5:].unique()

In [110]:
prices_agg = compute_quarter_metrics(prices, dates) 
prices_agg_melt = prices_agg.melt(id_vars=['Ticker','Date'])
prices_agg_melt.columns = ['Ticker', 'Date', 'Variable', 'Value']
prices_agg_melt.tail(1)

,Ticker,Date,Variable,Value
58589045,ZTS,2025-05-12,MACD_Hist,1.540961


In [111]:
sp500_merged = pd.concat([financials_fiscal, prices_agg_melt], axis=0)
sp500_merged.tail(1)

,Ticker,Date,Variable,Value
58589045,ZTS,2025-05-12,MACD_Hist,1.540961


In [112]:
sp500_merged['Value'] = clean_numeric_column(sp500_merged['Value'])
sp500_merged['Date']= sp500_merged['Date'].astype('str').str[:10]

In [113]:
sp500_wide = sp500_merged[['Ticker', 'Date', 'Variable', 'Value']].pivot_table(index=["Ticker", "Date"], columns="Variable", values="Value")
sp500_wide.reset_index(inplace=True)
sp500_wide.tail(1)

Variable,Ticker,Date,AFFO Per Share,Accounts Payable,Accounts Receivable,Accrued Expenses,Accrued Interest Payable,Accrued Interest Receivable,Acquisition of Real Estate Assets,Additional Paid-In Capital,Adjusted Funds From Operations (AFFO),Advertising Expenses,Allowance for Borrowed Funds for Construction,Allowance for Equity Funds for Construction,Allowance for Loan Losses,Amortization of Goodwill & Intangibles,Asset Management Fee,Asset Turnover,Asset Writedown,Asset Writedown & Restructuring Costs,Basic Shares Outstanding,Book Value Per Share,Brokerage Commission,Buildings,Buyback Yield / Dilution,Capital Expenditures,Cash & Equivalents,Cash & Short-Term Investments,Cash Acquisition,Cash Acquisitions,Cash Growth,Cash Income Tax Paid,Cash Interest Paid,Change in Accounts Payable,Change in Accounts Receivable,Change in Deferred Taxes,Change in Income Taxes,Change in Insurance Reserves / Liabilities,Change in Inventory,Change in Net Working Capital,Change in Other Net Operating Assets,Change in Trading Asset Securities,Change in Unearned Revenue,ClosePrice,Commissions and Fees,Common & Preferred Dividends Paid,Common Dividends Paid,Common Stock,Comprehensive Income & Other,Construction In Progress,Contributions to Nuclear Demissioning Trust,Cost of Revenue,Cost of Services Provided,CountDividends,Currency Exchange Gain (Loss),Currency Exchange Gains,Current Income Taxes Payable,Current Portion of Leases,Current Portion of Long-Term Debt,Current Ratio,Current Unearned Revenue,D&A For EBITDA,D&A For Ebitda,Debt / EBITDA Ratio,Debt / Equity Ratio,Debt / FCF Ratio,Deferred Long-Term Charges,Deferred Long-Term Tax Assets,Deferred Policy Acquisition Cost,Depreciation & Amortization,"Depreciation & Amortization, Total",Diluted Shares Outstanding,Distributions in Excess of Earnings,Divestitures,Dividend Growth,Dividend Per Share,Dividend Yield,Dividends Paid,EBIT,EBIT Margin,EBITDA,EBITDA Margin,EBT Excluding Unusual Items,EPS (Basic),EPS (Diluted),EPS Growth,EV/EBIT Ratio,EV/EBITDA Ratio,EV/FCF Ratio,EV/Sales Ratio,Earnings From Continuing Operations,Earnings From Continuing Ops.,Earnings From Discontinued Operations,Earnings From Discontinued Ops.,Earnings From Equity Investments,Earnings Yield,Effective Tax Rate,Enterprise Value,FCF Yield,FFO Payout Ratio,FFO Per Share,Federal Deposit Insurance,"Federal Home Loan Bank Debt, Long-Term",Filing Date Shares Outstanding,Financing Cash Flow,Fiscal year,Foreign Exchange Rate Adjustments,Forward PE,Free Cash Flow,Free Cash Flow Growth,Free Cash Flow Margin,Free Cash Flow Per Share,Fuel & Purchased Power,Funds From Operations (FFO),Gain (Loss) On Sale of Investments,Gain (Loss) on Sale of Assets,Gain (Loss) on Sale of Assets (Rev),Gain (Loss) on Sale of Equity Investments,Gain (Loss) on Sale of Investments,Gain (Loss) on Sale of Investments (Rev),Gain on Sale of Investments (Rev),Gain on Sale of Loans & Receivables,Goodwill,Gross Loans,Gross Margin,Gross Profit,Impairment of Goodwill,Income (Loss) Equity Investments,Income (Loss) on Equity Investments,Income From Trading Activities,Income Tax Expense,Institutional Deposits,Insurance & Annuity Liabilities,Insurance Settlements,Interest & Investment Income,Interest Bearing Deposits,Interest Expense,Interest Income,Interest Income on Investments,Interest Income on Loans,Interest Paid on Borrowings,Interest Paid on Deposits,Interest and Dividend Income,Inventory,Inventory Turnover,Investing Cash Flow,Investment In Debt and Equity Securities,Investment Securities,Investment in Marketable & Equity Securities,Investment in Securities,Investments in Debt & Equity Securities,Investments in Debt Securities,Investments in Equity & Preferred Securities,Investments in Real Estate,Issuance of Common Stock,Issuance of Preferred Stock,Land,Last Close Price,Leasehold Improvements,Legal Settlements,Levered Free Cash Flow,Loans & Lease Receivables,Loans Held for Sale,Loans Receivable Current,Long-Term Accounts Receivable,Long-Term Debt,Long-Term Debt Issued

In [114]:
sp500_wide_clean = sp500_wide.dropna(subset=['Shareholders\' Equity', 'ClosePrice'])
sp500_df = sp500_wide_clean.fillna(0)

In [115]:
sp500_df_names = sp500_df.merge(names[['Symbol', 'GICS Sector', 'GICS Sub-Industry', 'Founded']], how='left', left_on='Ticker', right_on='Symbol')

In [116]:
sp500_df_names.tail(1)

,Ticker,Date,AFFO Per Share,Accounts Payable,Accounts Receivable,Accrued Expenses,Accrued Interest Payable,Accrued Interest Receivable,Acquisition of Real Estate Assets,Additional Paid-In Capital,Adjusted Funds From Operations (AFFO),Advertising Expenses,Allowance for Borrowed Funds for Construction,Allowance for Equity Funds for Construction,Allowance for Loan Losses,Amortization of Goodwill & Intangibles,Asset Management Fee,Asset Turnover,Asset Writedown,Asset Writedown & Restructuring Costs,Basic Shares Outstanding,Book Value Per Share,Brokerage Commission,Buildings,Buyback Yield / Dilution,Capital Expenditures,Cash & Equivalents,Cash & Short-Term Investments,Cash Acquisition,Cash Acquisitions,Cash Growth,Cash Income Tax Paid,Cash Interest Paid,Change in Accounts Payable,Change in Accounts Receivable,Change in Deferred Taxes,Change in Income Taxes,Change in Insurance Reserves / Liabilities,Change in Inventory,Change in Net Working Capital,Change in Other Net Operating Assets,Change in Trading Asset Securities,Change in Unearned Revenue,ClosePrice,Commissions and Fees,Common & Preferred Dividends Paid,Common Dividends Paid,Common Stock,Comprehensive Income & Other,Construction In Progress,Contributions to Nuclear Demissioning Trust,Cost of Revenue,Cost of Services Provided,CountDividends,Currency Exchange Gain (Loss),Currency Exchange Gains,Current Income Taxes Payable,Current Portion of Leases,Current Portion of Long-Term Debt,Current Ratio,Current Unearned Revenue,D&A For EBITDA,D&A For Ebitda,Debt / EBITDA Ratio,Debt / Equity Ratio,Debt / FCF Ratio,Deferred Long-Term Charges,Deferred Long-Term Tax Assets,Deferred Policy Acquisition Cost,Depreciation & Amortization,"Depreciation & Amortization, Total",Diluted Shares Outstanding,Distributions in Excess of Earnings,Divestitures,Dividend Growth,Dividend Per Share,Dividend Yield,Dividends Paid,EBIT,EBIT Margin,EBITDA,EBITDA Margin,EBT Excluding Unusual Items,EPS (Basic),EPS (Diluted),EPS Growth,EV/EBIT Ratio,EV/EBITDA Ratio,EV/FCF Ratio,EV/Sales Ratio,Earnings From Continuing Operations,Earnings From Continuing Ops.,Earnings From Discontinued Operations,Earnings From Discontinued Ops.,Earnings From Equity Investments,Earnings Yield,Effective Tax Rate,Enterprise Value,FCF Yield,FFO Payout Ratio,FFO Per Share,Federal Deposit Insurance,"Federal Home Loan Bank Debt, Long-Term",Filing Date Shares Outstanding,Financing Cash Flow,Fiscal year,Foreign Exchange Rate Adjustments,Forward PE,Free Cash Flow,Free Cash Flow Growth,Free Cash Flow Margin,Free Cash Flow Per Share,Fuel & Purchased Power,Funds From Operations (FFO),Gain (Loss) On Sale of Investments,Gain (Loss) on Sale of Assets,Gain (Loss) on Sale of Assets (Rev),Gain (Loss) on Sale of Equity Investments,Gain (Loss) on Sale of Investments,Gain (Loss) on Sale of Investments (Rev),Gain on Sale of Investments (Rev),Gain on Sale of Loans & Receivables,Goodwill,Gross Loans,Gross Margin,Gross Profit,Impairment of Goodwill,Income (Loss) Equity Investments,Income (Loss) on Equity Investments,Income From Trading Activities,Income Tax Expense,Institutional Deposits,Insurance & Annuity Liabilities,Insurance Settlements,Interest & Investment Income,Interest Bearing Deposits,Interest Expense,Interest Income,Interest Income on Investments,Interest Income on Loans,Interest Paid on Borrowings,Interest Paid on Deposits,Interest and Dividend Income,Inventory,Inventory Turnover,Investing Cash Flow,Investment In Debt and Equity Securities,Investment Securities,Investment in Marketable & Equity Securities,Investment in Securities,Investments in Debt & Equity Securities,Investments in Debt Securities,Investments in Equity & Preferred Securities,Investments in Real Estate,Issuance of Common Stock,Issuance of Preferred Stock,Land,Last Close Price,Leasehold Improvements,Legal Settlements,Levered Free Cash Flow,Loans & Lease Receivables,Loans Held for Sale,Loans Receivable Current,Long-Term Accounts Receivable,Long-Term Debt,Long-Term Debt Issued,Long-Te

In [117]:
sp500_diff = calc_quarterly_pct_diff(sp500_df_names, ticker_col='Ticker', date_col='Date')
sp500_diff.dropna(subset=['ClosePrice_pct_diff_1', 'ClosePrice_pct_diff_4'], inplace = True)
sp500_diff.fillna(0, inplace = True)
sp500_diff.drop(columns=['Symbol'], axis=1, inplace = True)
sp500_diff['Founded'] = sp500_diff['Founded'].str[0:4].astype('int')
sp500_diff.head()

,Ticker,Date,AFFO Per Share,Accounts Payable,Accounts Receivable,Accrued Expenses,Accrued Interest Payable,Accrued Interest Receivable,Acquisition of Real Estate Assets,Additional Paid-In Capital,Adjusted Funds From Operations (AFFO),Advertising Expenses,Allowance for Borrowed Funds for Construction,Allowance for Equity Funds for Construction,Allowance for Loan Losses,Amortization of Goodwill & Intangibles,Asset Management Fee,Asset Turnover,Asset Writedown,Asset Writedown & Restructuring Costs,Basic Shares Outstanding,Book Value Per Share,Brokerage Commission,Buildings,Buyback Yield / Dilution,Capital Expenditures,Cash & Equivalents,Cash & Short-Term Investments,Cash Acquisition,Cash Acquisitions,Cash Growth,Cash Income Tax Paid,Cash Interest Paid,Change in Accounts Payable,Change in Accounts Receivable,Change in Deferred Taxes,Change in Income Taxes,Change in Insurance Reserves / Liabilities,Change in Inventory,Change in Net Working Capital,Change in Other Net Operating Assets,Change in Trading Asset Securities,Change in Unearned Revenue,ClosePrice,Commissions and Fees,Common & Preferred Dividends Paid,Common Dividends Paid,Common Stock,Comprehensive Income & Other,Construction In Progress,Contributions to Nuclear Demissioning Trust,Cost of Revenue,Cost of Services Provided,CountDividends,Currency Exchange Gain (Loss),Currency Exchange Gains,Current Income Taxes Payable,Current Portion of Leases,Current Portion of Long-Term Debt,Current Ratio,Current Unearned Revenue,D&A For EBITDA,D&A For Ebitda,Debt / EBITDA Ratio,Debt / Equity Ratio,Debt / FCF Ratio,Deferred Long-Term Charges,Deferred Long-Term Tax Assets,Deferred Policy Acquisition Cost,Depreciation & Amortization,"Depreciation & Amortization, Total",Diluted Shares Outstanding,Distributions in Excess of Earnings,Divestitures,Dividend Growth,Dividend Per Share,Dividend Yield,Dividends Paid,EBIT,EBIT Margin,EBITDA,EBITDA Margin,EBT Excluding Unusual Items,EPS (Basic),EPS (Diluted),EPS Growth,EV/EBIT Ratio,EV/EBITDA Ratio,EV/FCF Ratio,EV/Sales Ratio,Earnings From Continuing Operations,Earnings From Continuing Ops.,Earnings From Discontinued Operations,Earnings From Discontinued Ops.,Earnings From Equity Investments,Earnings Yield,Effective Tax Rate,Enterprise Value,FCF Yield,FFO Payout Ratio,FFO Per Share,Federal Deposit Insurance,"Federal Home Loan Bank Debt, Long-Term",Filing Date Shares Outstanding,Financing Cash Flow,Fiscal year,Foreign Exchange Rate Adjustments,Forward PE,Free Cash Flow,Free Cash Flow Growth,Free Cash Flow Margin,Free Cash Flow Per Share,Fuel & Purchased Power,Funds From Operations (FFO),Gain (Loss) On Sale of Investments,Gain (Loss) on Sale of Assets,Gain (Loss) on Sale of Assets (Rev),Gain (Loss) on Sale of Equity Investments,Gain (Loss) on Sale of Investments,Gain (Loss) on Sale of Investments (Rev),Gain on Sale of Investments (Rev),Gain on Sale of Loans & Receivables,Goodwill,Gross Loans,Gross Margin,Gross Profit,Impairment of Goodwill,Income (Loss) Equity Investments,Income (Loss) on Equity Investments,Income From Trading Activities,Income Tax Expense,Institutional Deposits,Insurance & Annuity Liabilities,Insurance Settlements,Interest & Investment Income,Interest Bearing Deposits,Interest Expense,Interest Income,Interest Income on Investments,Interest Income on Loans,Interest Paid on Borrowings,Interest Paid on Deposits,Interest and Dividend Income,Inventory,Inventory Turnover,Investing Cash Flow,Investment In Debt and Equity Securities,Investment Securities,Investment in Marketable & Equity Securities,Investment in Securities,Investments in Debt & Equity Securities,Investments in Debt Securities,Investments in Equity & Preferred Securities,Investments in Real Estate,Issuance of Common Stock,Issuance of Preferred Stock,Land,Last Close Price,Leasehold Improvements,Legal Settlements,Levered Free Cash Flow,Loans & Lease Receivables,Loans Held for Sale,Loans Receivable Current,Long-Term Accounts Receivable,Long-Term Debt,Long-Term Debt Issued,Long-Te

In [118]:
sp500_diff.to_csv('sp500_diff.csv')

In [121]:
#compute share of zeros for each columns
sp500_diff.isin([0]).mean() * 100

Ticker                                  0.000000
Date                                    0.000000
AFFO Per Share                         95.900797
Accounts Payable                        8.031354
Accounts Receivable                    37.599589
                                         ...    
Unlevered Free Cash Flow_pct_diff_4     6.643536
Unpaid Claims_pct_diff_1               96.093549
Unpaid Claims_pct_diff_4               96.067849
Working Capital_pct_diff_1             14.006682
Working Capital_pct_diff_4             14.032382
Length: 1133, dtype: float64